# Assignment of peer feedback

This notebook contains the code used to determine which two other teams each team must give feed back to in the data project

In [ ]:
from github import Github # pip install PyGithub
import sys
import datetime
import pandas as pd
import numpy as np
import random
from collections import Counter

In [ ]:
# load personal access token
with open("C:/Users/gmf123/Dropbox/github_token.txt", mode = "r") as file:
    token = file.read()

year = "2023"    
class_name = "projects-" + year    

### Load all repositories in this year's class room

In [ ]:
# a. access github through access token
gh = Github(token)
org = gh.get_organization('NumEconCopenhagen')
all_repos = org.get_repos()

# b. locate all repositories in current class 
class_name = "projects-" + year
current_class = [repo.name for repo in all_repos if class_name in repo.name]

### All teams

In [ ]:
disregard_repos = [] # manually add

In [ ]:
teams = set()
i = 1
for repo in all_repos:
    if repo.name not in current_class: continue
    if repo.name[14:] in disregard_repos: 
        print(repo.name)
        continue 
    teams.add(repo.name[14:])
    i += 1

### Excel sheets

In [ ]:
random.seed(123)
for k in range(10):

    try:

        for projectname in ['inauguralproject', 'dataproject', 'modelproject']:

            # a. create data frame
            teams_df = pd.DataFrame(data=teams,columns=['team'])
            teams_df.sort_values(by='team',inplace=True)
            teams_df.reset_index(drop=True, inplace=True)
            teams_df['peer_group_1'] = np.nan
            teams_df['peer_group_2'] = np.nan

            # b. assign peers to teams
            N = len(teams_df.team.values)
            counter = [0 for _ in range(N)]
            assigned = []

            # for each team, loop over teams that have not already been assigned
            for i,_ in enumerate(teams_df.team):
                pop = [t for t in range(N) if (t != i) & (t not in assigned)]
                peers = random.sample(pop, 2)    
                for c in [0,1]:
                    teams_df.iloc[i, 1 + c] = teams_df.team[peers[c]]
                    counter[peers[c]] += 1
                    if counter[peers[c]] == 2:
                        assigned.append(peers[c])

            teams_df.to_excel(f'PeerAssignment_{projectname}.xlsx',index=False)

        print(f'attempt {k+1} was successful')
        break

    except Exception as e:

        print(f'attempt {k+1} failed: {e}')